In [1]:
import os

from google.colab import drive
drive.mount('/content/gdrive')

os.chdir('/content/gdrive/MyDrive/first_try_of_fastai')

print("------------------------------------------------------------------")

Mounted at /content/gdrive
------------------------------------------------------------------


In [2]:
#export
import os
os.chdir('/content/gdrive/MyDrive/first_try_of_fastai/exp')
from nb_05b import *
os.chdir('/content/gdrive/MyDrive/first_try_of_fastai')

**ConvNet**

In [3]:
x_train, y_train, x_valid, y_valid = get_data()

In [4]:
#export
def normalize_to(train, valid):
  m, s = train.mean(), train.std()
  return normalize(train, m, s), normalize(valid, m, s)

In [5]:
x_train, x_valid = normalize_to(x_train, x_valid)
train_ds,valid_ds = Dataset(x_train, y_train), Dataset(x_valid, y_valid)

In [6]:
x_train.mean(),x_train.std()

(tensor(3.8966e-08), tensor(1.))

In [7]:
nh,bs = 50,512
c = y_train.max().item()+1
loss_func = F.cross_entropy

data = DataBunch(*get_dls(train_ds, valid_ds, bs), c)

In [8]:
#eport
class Lambda(nn.Module):
  def __init__(self, func):
    super().__init__()
    self.func = func

  def forward(self, x): return self.func(x)

def flatten(x): return x.view(x.shape[0], -1)

In [9]:
def mnist_resize(x): return x.view(-1,1,28,28)

In [10]:
def get_cnn_model(data):
  return nn.Sequential(
      Lambda(mnist_resize),
      nn.Conv2d( 1, 8,5, padding=2, stride=2), nn.ReLU(),
      nn.Conv2d( 8,16,3, padding=1, stride=2), nn.ReLU(),
      nn.Conv2d(16,32,3, padding=1, stride=2), nn.ReLU(),
      nn.Conv2d(32,32,3, padding=1, stride=2), nn.ReLU(),
      nn.AdaptiveAvgPool2d(1),
      Lambda(flatten),
      nn.Linear(32, data.c)
  )

In [11]:
model = get_cnn_model(data)

In [12]:
#Copy this to any notebook complaining about recorder or statscallbacks
class AvgStats():
  def __init__(self, metrics, in_train):
    self.metrics = listify(metrics)
    self.in_train = in_train
    self.tot_loss = 0.
    self.count=0
    self.tot_mets = [0.] * len(self.metrics)

  def reset(self):
    self.tot_loss, self.count = 0.,0
    self.tot_mets = [0.] * len(self.metrics)
  
  @property
  def all_stats(self): return[self.tot_loss.item()] + self.tot_mets
  @property
  def avg_stats(self): return[o/self.count for o in self.all_stats]

  def __repr__(self):
    if not self.count: return ''
    return f"{'train' if self.in_train else 'valid'}: {self.avg_stats}"

  def accumulate(self, run):
    bn = run.xb.shape[0]
    self.tot_loss += run.loss * bn
    self.count += bn
    for i,m in enumerate(self.metrics):
      self.tot_mets[i] += m(run.pred, run.yb) * bn

class AvgStatsCallback(Callback):
  def __init__(self, metrics):
    self.train_stats,self.valid_stats = AvgStats(metrics,True), AvgStats(metrics, False)

  def begin_epoch(self):
    self.train_stats.reset()
    self.valid_stats.reset()

  def after_loss(self):
    stats = self.train_stats if self.in_train else self.valid_stats
    with torch.no_grad(): stats.accumulate(self.run)

  def after_epoch(self):
    print(self.train_stats)
    print(self.valid_stats)

class Recorder(Callback):
  def begin_fit(self): self.lrs, self.losses = [],[]

  def after_batch(self):
    if not self.in_train: return
    self.lrs.append(self.opt.param_groups[-1]['lr'])
    self.losses.append(self.loss.detach().cpu())

  def plot_lr(self): plt.plot(self.lrs)
  def plot_loss(self): plt.plot(self.losses)

In [13]:
cbfs = [Recorder, partial(AvgStatsCallback, accuracy)]

In [14]:
opt = optim.SGD(model.parameters(), lr = 0.4)
learn = Learner(model, opt, loss_func, data)
run = Runner(cb_funcs=cbfs)

In [15]:
%time run.fit(1, learn)

train: [2.0138684375, tensor(0.3084)]
valid: [1.07881845703125, tensor(0.6118)]
CPU times: user 3.91 s, sys: 122 ms, total: 4.03 s
Wall time: 4.23 s


**CUDA**

In [16]:
device = torch.device('cuda',0)

In [17]:
class CudaCallback(Callback):
  def __init__(self,device): self.device=device
  def begin_fit(self): self.model.to(device)
  def begin_batch(self): self.run.xb,self.run.yb = self.xb.to(device),self.yb.to(device)

In [18]:
torch.cuda.set_device(device)

In [19]:
class CudaCallback(Callback):
  def begin_fit(self): self.model.cuda()
  def begin_batch(self): self.run.xb,self.run.yb = self.xb.cuda(),self.yb.cuda()

In [20]:
cbfs.append(CudaCallback)

In [21]:
model = get_cnn_model(data)

In [22]:
opt = optim.SGD(model.parameters(), lr=0.4)
learn = Learner(model, opt, loss_func, data)
run = Runner(cb_funcs=cbfs)

In [23]:
%time run.fit(3, learn)

train: [2.20921453125, tensor(0.2194, device='cuda:0')]
valid: [1.23999736328125, tensor(0.6801, device='cuda:0')]
train: [2.20921453125, tensor(0.2194, device='cuda:0')]
valid: [1.26274765625, tensor(0.6663, device='cuda:0')]
train: [2.20921453125, tensor(0.2194, device='cuda:0')]
valid: [1.2644978365384616, tensor(0.6653, device='cuda:0')]
CPU times: user 3.72 s, sys: 1.22 s, total: 4.93 s
Wall time: 12.2 s


**Refactor**

In [ ]:
# left it at 1.03.xx lesson 10 (2019): looking inside the model